# Model Building

## Introduction

`Model building is the core requirment here! and we have build the model which stisfy the requirments..lets first look the problem stament..`

**Problem Statement**:-
Twitter has now become a useful way to build one's business as it helps in giving the brand a voice and a personality. The platform is also a quick, easy and inexpensive way to gain valuable insight from the desired audience. Identifying the sentiments about the product/brand can help the business take better actions.

You have with you evaluated tweets about multiple brands. The evaluators(random audience) were asked if the tweet expressed positive, negative, or no emotion towards a product/brand and labelled accordingly.

**`from the problem statment we can understand that,to build the model which can predict the sentiments of tweet of diffrent products behalf of random audiance`**

So we have **train data** ,from this we have to train the model..

`lets find which **ML** will give best evaluation matrics score as well more stabl?`

**Evaluation Metric**
-`F1 score` 

**F1 score**: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

F1 = 2 * (precision * recall) / (precision + recall)

# Necessary Library

In [175]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import sklearn
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

from sklearn.model_selection import KFold,StratifiedKFold,cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.metrics import f1_score,accuracy_score

In [2]:
train = pd.read_csv("treated_train.csv").set_index("tweet_id")
test = pd.read_csv("treated_test.csv").set_index("tweet_id")

In [3]:
pd.set_option('max_colwidth',150)
train.head(10)

,tweet,sentiment,tb_polarity,tb_subjectivity,hashtags
tweet_id,,,,,
1701,defining language of touch with different dialects becoming smaller,1,0.150000,0.650000,sxswnui sxsw apple
1851,learning ab google doodles all doodles should be light funny innovative with exceptions for significant occasions,1,0.381250,0.893750,googledoodle sxsw
2689,one of the most inyourface ex of stealing the show in years rt at apple schools the market experts,2,0.500000,0.500000,sxsw
4525,this iphone application would be pretty awesome if it did not crash every mins during extended browsing,0,0.625000,1.000000,sxsw fuckit illmakeitwork
3604,line outside the apple store in austin waiting for the new ipad,1,0.068182,0.252273,sxsw
966,one lone dude awaits ipad at apples sxsw store,1,0.000000,0.000000,technews technews apple ipad sxsw tablets tech
1395,sxsw tips prince npr videos toy shopping with zuckerberg,1,0.000000,0.000000,sxsw ipad
8182,nu user rt new for now in the application store includes uberguide to sponsored by,1,0.136364,0.454545,ubersocial iphone sxsw mashable
8835,free sampler on itunes,2,0.400000,0.800000,sxsw freemusic


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7274 entries, 1701 to 3162
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet            7264 non-null   object 
 1   sentiment        7274 non-null   int64  
 2   tb_polarity      7274 non-null   float64
 3   tb_subjectivity  7274 non-null   float64
 4   hashtags         7268 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 341.0+ KB


In [4]:
stop_words = list(set(stopwords.words('english')))+list(punctuation)
train['tweet'] = train['tweet'].apply(lambda x:word_tokenize(str(x)))
train['tweet'] = train['tweet'].apply(lambda row:[word for word in row if word not in stop_words])

test['tweet'] = test['tweet'].apply(lambda x:word_tokenize(str(x)))
test['tweet'] = test['tweet'].apply(lambda row:[word for word in row if word not in stop_words])

In [5]:
lemma = WordNetLemmatizer()
train['tweet'] = train['tweet'].apply(lambda x:[lemma.lemmatize(i) for i in x])

test['tweet'] = test['tweet'].apply(lambda x:[lemma.lemmatize(i) for i in x])

In [6]:
train['tweet'] = train['tweet'].apply(lambda x:' '.join(x))
test['tweet'] = test['tweet'].apply(lambda x:' '.join(x))

In [150]:
cv = CountVectorizer()
text = train['tweet']
vector = cv.fit_transform(text)
X = vector.toarray()

In [14]:
text = train['tweet']
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(text)
X = vector.toarray()

In [151]:
y = train['sentiment']

In [152]:
def model_scores(y_test,y_pred):
    print("F1 score is : ",f1_score(y_test,y_pred,average="weighted"),"\n Accuracy is : ",accuracy_score(y_test,y_pred))

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.4,random_state=0)
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.2,random_state=0)

In [153]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

## Oversampling

In [117]:
smote = SMOTE(random_state=0)

In [118]:
X_train,y_train = smote.fit_sample(X_train,y_train)

In [119]:
logreg = LogisticRegression(max_iter=250,random_state=0)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.5948166625902398 
 Accuracy is :  0.5803939532753092


In [120]:
lsvm = LinearSVC(random_state=0)
lsvm.fit(X_train,y_train)
y_pred = lsvm.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.6140266780947191 
 Accuracy is :  0.602840128263857


In [121]:
ros = RandomOverSampler(random_state=0)
X_train,y_train = ros.fit_sample(X_train,y_train)

In [122]:
logreg = LogisticRegression(max_iter=250,random_state=0)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.5948166625902398 
 Accuracy is :  0.5803939532753092


In [123]:
lsvm = LinearSVC(random_state=0)
lsvm.fit(X_train,y_train)
y_pred = lsvm.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.6140266780947191 
 Accuracy is :  0.602840128263857


## Modelling

## Logestic Regression

In [154]:
logreg = LogisticRegression(max_iter=150,random_state=0)

In [155]:
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=150,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [156]:
y_pred = logreg.predict(X_test)

In [157]:
model_scores(y_test,y_pred)

F1 score is :  0.6539121846522623 
 Accuracy is :  0.6697205680256527


In [177]:
ovr = OneVsRestClassifier(estimator=logreg,n_jobs=-1)

In [178]:
ovr.fit(X_train,y_train)
y_pred = ovr.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.6507231041278736 
 Accuracy is :  0.6710948236371965


## Random Forest

In [85]:
rf = RandomForestClassifier(random_state=0)

In [86]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [87]:
y_pred = rf.predict(X_test)

In [88]:
model_scores(y_test,y_pred)

0.6369117882381689 0.663307375171782


## Naive_Bayes

In [125]:
nb = MultinomialNB(alpha=0.8)

In [126]:
nb.fit(X_train,y_train)

MultinomialNB(alpha=0.8, class_prior=None, fit_prior=True)

In [127]:
y_pred = nb.predict(X_test)

In [128]:
model_scores(y_test,y_pred)

F1 score is :  0.6372844518844325 
 Accuracy is :  0.6491067338524965


## SVM

In [167]:
lsvm = LinearSVC(random_state=0)

In [168]:
lsvm.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [169]:
y_pred = lsvm.predict(X_test)

In [170]:
model_scores(y_test,y_pred)

F1 score is :  0.6565603539922594 
 Accuracy is :  0.6642235455794778


## Xgboost

In [133]:
xg = XGBClassifier(random_state=0,n_jobs=-1,verbose=1)

In [134]:
xg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbose=1, verbosity=1)

In [135]:
y_pred = xg.predict(X_test)

In [136]:
model_scores(y_test,y_pred)

F1 score is :  0.5779693781996347 
 Accuracy is :  0.6491067338524965


In [244]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

## Cross Validation

- we are going to cheak the variance of model which gives idea to which model is more stable

In [60]:
kfold = KFold(n_splits=10)

In [65]:
for train_index,test_index in kfold.split(X,y):
    #print("Train:",train_index,"Validation:",test_index)
    X_train, X_test = X[train_index], X[test_index]

In [74]:
skf = StratifiedKFold(n_splits=10)

In [68]:
for train_index,test_index in skf.split(X,y):
    #print("Train:",train_index,"Validation:",test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [69]:
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
model_scores(y_test,y_pred)

0.6507233295761533 0.66996699669967


In [234]:
scorer = make_scorer()

In [80]:
score = cross_val_score(logreg,X_train,y_train,cv=skf,scoring='f1_weighted',verbose=1,n_jobs=-1)
score

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  2.0min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.2min finished


array([0.64985085, 0.63020541, 0.64509991, 0.63592612, 0.63237173,
       0.66205328, 0.63336122, 0.63552704, 0.64248448, 0.61187254])

In [81]:
np.var(score)

0.0001596379013577824

In [86]:
score = cross_val_score(lsvm,X_train,y_train,cv=skf,scoring='f1_weighted',verbose=1,n_jobs=-1)
score

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    7.8s remaining:    5.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.5s finished


array([0.65237188, 0.62811556, 0.65116306, 0.63107005, 0.62667302,
       0.64442248, 0.64067055, 0.626503  , 0.62824004, 0.59610111])

In [87]:
np.var(score)

0.0002386305802547271

In [77]:
score = cross_val_score(rf,X_train,y_train,cv=skf,scoring='f1_weighted',verbose=1,n_jobs=3)
score

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:   52.4s finished


array([0.48325556, 0.46266534, 0.45510136, 0.47212653, 0.46028623,
       0.48206789, 0.45271495, 0.50616161, 0.47927025, 0.47212013])

In [79]:
np.var(score)

0.0002326448634042435

## Hyperparameter Tuning

In [15]:
lsvm = LinearSVC(random_state=0)
parameter = {"penalty":["l1","l2"],
            "C":[0,0.01,0.1,0.8,1,1.2],
            "max_iter":[150,300,500,1000]}

In [72]:
random = RandomizedSearchCV(estimator=lsvm,param_distributions=parameter,scoring="f1_weighted",verbose=1,n_jobs=-1)

In [73]:
random.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   41.8s finished


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                       fit_intercept=True, intercept_scaling=1,
                                       loss='squared_hinge', max_iter=1000,
                                       multi_class='ovr', penalty='l2',
                                       random_state=0, tol=0.0001, verbose=0),
                   iid='deprecated', n_iter=40, n_jobs=-1,
                   param_distributions={'C': [0, 0.01, 0.1, 0.8, 1, 1.2],
                                        'max_iter': [150, 300, 500, 1000],
                                        'penalty': ['l1', 'l2']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='f1_weighted', verbose=1)

In [137]:
random.best_params_,random.best_score_

({'n_estimators': 100,
  'max_features': 100,
  'max_depth': 20,
  'criterion': 'entropy',
  'bootstrap': 'False'},
 0.47039969617540534)

In [75]:
lsvm = LinearSVC(random_state=0,penalty='l2',max_iter=1000,C=1)

In [76]:
lsvm.fit(X_train,y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [77]:
y_pred = lsvm.predict(X_test)

In [78]:
model_scores(y_test,y_pred)

0.6505930441963678 0.6651397159871736


In [16]:
grid = GridSearchCV(estimator=lsvm,param_grid=parameter,scoring="f1_weighted",n_jobs=-1,cv=10,verbose=1)

In [17]:
grid.fit(X_train,y_train)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  2.2min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=0, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0, 0.01, 0.1, 0.8, 1, 1.2],
                         'max_iter': [150, 300, 500, 1000],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=1)

In [19]:
grid.best_params_,grid.best_score_

({'C': 0.1, 'max_iter': 150, 'penalty': 'l2'}, 0.6391940358756075)

In [143]:
lsvm = LinearSVC(random_state=0,penalty='l2',max_iter=150,C=0.1)
lsvm.fit(X_train,y_train)
y_pred = lsvm.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.6538828622191672 
 Accuracy is :  0.6729271644525882


In [138]:
rf = RandomForestClassifier()
parameters = {"n_estimators":[10,100],
              'criterion':['gini','entropy'],
             'max_depth':[20,60,'None','sqrt'],
            "max_features":[10,100,200],
             'bootstrap':['True','False']}

In [139]:
random = RandomizedSearchCV(estimator=rf,param_distributions=parameters,scoring="f1_weighted",verbose=1,n_jobs=-1)

In [140]:
random.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.2min finished


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [141]:
random.best_params_,random.best_score_

({'n_estimators': 100,
  'max_features': 200,
  'max_depth': 60,
  'criterion': 'entropy',
  'bootstrap': 'False'},
 0.5959090411044926)

In [142]:
rf = RandomForestClassifier(n_estimators= 100,
 max_features= 200,
 max_depth= 60,
 criterion= 'entropy',
 bootstrap= 'False')
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
model_scores(y_test,y_pred)

F1 score is :  0.5961555090325988 
 Accuracy is :  0.6504809894640403


# **After training diffrent models we get the best model for this data is `Logestic Regression` due to very less variance and best fit comapre to other models**

In [179]:
text = test['tweet']
vector = cv.transform(text)
X = vector.toarray()

In [184]:
y_pred = logreg.predict(X)

In [185]:
sample_submission = pd.DataFrame(y_pred,test.index,columns=['sentiment'])

In [186]:
sample_submission.head()

,sentiment
tweet_id,
7506,1
7992,1
247,1
7688,2
3294,2


In [187]:
sample_submission.to_csv("sample_sub.csv")